In [1]:
import numpy as np
import integral_image_transform
import sklearn.datasets
import math

def get_faces():
    return sklearn.datasets.fetch_olivetti_faces()

images = get_faces()['images']
labels = get_faces()['target']

In [2]:
def integral_transform(img, square = True, ravelled = False, dims = None):
    if square and ravelled:
        side_len = int(len(img) ** .5)
        img_reshaped = np.reshape(img, (side_len, side_len))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img_reshaped[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img_reshaped[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img_reshaped[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img_reshaped[j][i-j]
    if square and not ravelled:
        side_len = int(len(img))
        ans = np.zeros((side_len, side_len))
        for i in range(2*side_len - 1):
            for j in range(i + 1):
                if j < side_len and i - j < side_len:
                    if i == 0:
                        ans[0][0] = img[0][0]
                    elif j == 0:
                        ans[j][i-j] = ans[j][i-j-1] + img[j][i-j]
                    elif i == j:
                        ans[j][i-j] = ans[j-1][i-j] + img[j][i-j]
                    else:
                        ans[j][i-j] = ans[j][i-j-1] + ans[j-1][i-j] - ans[j-1][i-j-1] + img[j][i-j]

    return ans




In [3]:
image_test = integral_transform(images[0])

In [4]:
def compute_two_rectangle_features(integral_image):
    # Calculating at every point and scale for now
    # TODO exchange height and width for some 24x24 window thing
        # to do this, probably just pass in a 24 x 24 window
    height,width = integral_image.shape # number of rows, number of columns
    vertical_features = np.zeros((height,width,height,width))
    horizontal_features = np.zeros((height,width,height,width))
    
    # Vertical (side by side) rectangles
    for i in range(width): # Starting x
        for j in range(height-2): # Starting y
            for k in range(1,int(math.floor((width-i)/2.0))): # Height of rectangles
                for z in range(1,height-j): # Width of rectangles
                    left = integral_image[i+k,j+z]- integral_image[i,j]
                    right = integral_image[i+2*k,j+z]- integral_image[i+k,j]
                    vertical_features[i,j,k,z] = left-right
                    
    # Horizontal (stacked) rectangles
    for i in range(width): # Starting x
        for j in range(height-2): # Starting y
            for k in range(1,width-i): # Height of rectangles
                for z in range(1,int(math.floor((height-j)/2.0))): # Width of rectangles
                    top = integral_image[i+k,j+z]-integral_image[i,j]
                    bottom = integral_image[i+k,j+2*z]-integral_image[i,j+z]
                    horizontal_features[i,j,k,z] = bottom-top
    
    return vertical_features,horizontal_features


vertical_features,horizontal_features = compute_two_rectangle_features(image_test)
# Takes 1.95 seconds this slow way

In [17]:
print vertical_features[(1,1,24,24)]
print horizontal_features[(0,0,30,30)]
# print image_test
# print three_rectangle_features != 0.0

-405.504132681
589.599174902
[[[[[False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]
    ..., 
    [False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]]

   [[False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]
    ..., 
    [False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]]

   [[False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]
    ..., 
    [False False False ..., False False False]
    [False False False ..., False False False]
    [False False False ..., False False False]]

   ..., 
   [[False False False ..., False False False]
    [False False False ..., False

In [9]:
##### JUNK doesn't work

def compute_three_rectangle_features(integral_image):
    # Calculating at every point and scale for now
    # Assuming that the dimensions of the outer two are the same
    
    height,width = integral_image.shape
    features = np.zeros((height,width,height,width,width)) # (top left x, top left y,\
                        # bottom right x, bottom right y, width of center rectange)
    
    # Vertical (side by side) rectangles
    for i in range(width-3): # Top left
        for j in range(height): # Top right
            for k in range(2,i): # Bottom right
                for z in range(1,j): # Bottom right
                    for c in range(k-z-2): # Width center
                        if (k-i-c) & 2 != 0:
                            pass
                        try:
                            left = integral_image[i+(k-i-c)/2,j+z]-integral_image[i,j]
                            center = integral_image[k-c,j+z]-integral_image[i+(k-i-c)/2,j]
                            right = integral_image[i+k,j+z]-integral_image[k-c,j]
                            features[i,j,k,z,c] = left+right-center
                        except IndexError:
                            pass
    
    return features

# TODO index errors out of range thing
    
three_rectangle_features = compute_three_rectangle_features(image_test)